In [2]:
import json
from collections import defaultdict


def convert_schedule(json_data):
    courses = defaultdict(list)

    for sala in json_data["salas"]:
        unidade = sala["unidade"]
        nome_sala = sala["nome"]

        for dia, horarios in sala["horarios"].items():
            for horario_info in horarios:
                horario = horario_info["horario"]
                aula = horario_info["aula"]

                if "Almoço" in aula:
                    continue

                siglas_cursos = []
                for prefix in ["CC", "ES", "EC", "EP", "EM"]:
                    if f"{prefix}" in aula.split()[0]:
                        siglas_cursos.append(prefix)

                if "/" in aula.split()[0]:
                    combined = aula.split()[0].split("/")
                    siglas_cursos = [
                        c for c in combined if c in ["CC", "ES", "EC", "EP", "EM"]
                    ]

                if not siglas_cursos:
                    continue

                parts = aula.split("(")
                nome_curso_completo = parts[0].strip()
                professor = "(" + parts[1] if len(parts) > 1 else ""

                nome_curso = nome_curso_completo
                for sigla in siglas_cursos:
                    nome_curso = nome_curso.replace(sigla, "")

                nome_curso = nome_curso.lstrip("0123456789/").strip()

                for sigla_curso in siglas_cursos:
                    curso_existente = None
                    for curso in courses[sigla_curso]:
                        if curso["name_course"] == nome_curso:
                            curso_existente = curso
                            break

                    if not curso_existente:
                        curso_existente = {
                            "name_course": nome_curso,
                            "week_day": defaultdict(dict),
                        }
                        courses[sigla_curso].append(curso_existente)

                    curso_existente["week_day"][dia][horario] = {
                        "Unidade": unidade,
                        "Sala": nome_sala,
                        "Prof(a)": professor,
                    }

    return dict(courses)


with open("generated_by_deepseek.json", "r", encoding="utf-8") as f:
    original_data = json.load(f)

new_format = convert_schedule(original_data)

with open("data.json", "w", encoding="utf-8") as f:
    json.dump(new_format, f, ensure_ascii=False, indent=4)

print("Conversão concluída! Verifique o arquivo 'data.json'")


Conversão concluída! Verifique o arquivo 'data.json'
